Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (10000, 28, 28) (10000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (10000, 28, 28, 1) (10000, 10)
Validation set (1000, 28, 28, 1) (1000, 10)
Test set (1000, 28, 28, 1) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [23]:
#GENERAR ESTRUCTURA DE CONVOLUTIONAL NEURAL NETWORK#

#Variables gloables
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    ### Input data ### - Igual que en anteriores ocasiones
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ### Variables ###
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    print('LAYER1 - Weights: ',layer1_weights)
    print('LAYER1 - Bias: ',layer1_biases,'\n')
    
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    print('LAYER2 - Weights: ',layer2_weights)
    print('LAYER2 - Bias: ',layer2_biases,'\n')
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    print('LAYER3 - Weights: ',layer3_weights)
    print('LAYER3 - Bias: ',layer3_biases,'\n')

    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    print('LAYER4 - Weights: ',layer4_weights)
    print('LAYER4 - Bias: ',layer4_biases,'\n')

    # Model.
    '''
    Let's build a small network with: 
        ->two convolutional layers
        ->one fully connected layer.
    '''
    def model(data):
        #1st Conv Layer
        conv = tf.nn.conv2d(input=data, filter=layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        print('Data: ',data)
        print('Conv1: ',conv)
        print('Hidden1: ',hidden,'\n')
        
        #2nd Conv Layer
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        print('Conv2: ',conv)
        print('Hidden2: ',hidden,'\n')
        
        
        #Fully connected layer
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        print('FullyConnected: ',hidden,'\n')
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

LAYER1 - Weights:  Tensor("Variable/read:0", shape=(5, 5, 1, 16), dtype=float32)
LAYER1 - Bias:  Tensor("Variable_1/read:0", shape=(16,), dtype=float32) 

LAYER2 - Weights:  Tensor("Variable_2/read:0", shape=(5, 5, 16, 16), dtype=float32)
LAYER2 - Bias:  Tensor("Variable_3/read:0", shape=(16,), dtype=float32) 

LAYER3 - Weights:  Tensor("Variable_4/read:0", shape=(784, 64), dtype=float32)
LAYER3 - Bias:  Tensor("Variable_5/read:0", shape=(64,), dtype=float32) 

LAYER4 - Weights:  Tensor("Variable_6/read:0", shape=(64, 10), dtype=float32)
LAYER4 - Bias:  Tensor("Variable_7/read:0", shape=(10,), dtype=float32) 

Data:  Tensor("Placeholder:0", shape=(16, 28, 28, 1), dtype=float32)
Conv1:  Tensor("Conv2D:0", shape=(16, 14, 14, 16), dtype=float32)
Hidden1:  Tensor("Relu:0", shape=(16, 14, 14, 16), dtype=float32) 

Conv2:  Tensor("Conv2D_1:0", shape=(16, 7, 7, 16), dtype=float32)
Hidden2:  Tensor("Relu_1:0", shape=(16, 7, 7, 16), dtype=float32) 

FullyConnected:  Tensor("Relu_2:0", shape=(16

In [24]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.085980
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.672895
Minibatch accuracy: 50.0%
Validation accuracy: 59.3%
Minibatch loss at step 100: 0.737033
Minibatch accuracy: 75.0%
Validation accuracy: 76.0%
Minibatch loss at step 150: 1.045159
Minibatch accuracy: 68.8%
Validation accuracy: 77.2%
Minibatch loss at step 200: 0.861703
Minibatch accuracy: 56.2%
Validation accuracy: 79.9%
Minibatch loss at step 250: 0.652469
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 300: 0.239698
Minibatch accuracy: 93.8%
Validation accuracy: 77.5%
Minibatch loss at step 350: 0.748839
Minibatch accuracy: 68.8%
Validation accuracy: 80.9%
Minibatch loss at step 400: 0.587268
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 450: 0.307097
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 500: 0.863820
Minibatch accuracy: 68.8%
Validation accuracy: 82.3%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [27]:
#GENERAR ESTRUCTURA DE CONVOLUTIONAL NEURAL NETWORK CON MAX POOLING#

#Variables gloables
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    ### Input data ### - Igual que en anteriores ocasiones
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    ### Variables ###
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    print('LAYER1 - Weights: ',layer1_weights)
    print('LAYER1 - Bias: ',layer1_biases,'\n')
    
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    print('LAYER2 - Weights: ',layer2_weights)
    print('LAYER2 - Bias: ',layer2_biases,'\n')
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    print('LAYER3 - Weights: ',layer3_weights)
    print('LAYER3 - Bias: ',layer3_biases,'\n')

    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    print('LAYER4 - Weights: ',layer4_weights)
    print('LAYER4 - Bias: ',layer4_biases,'\n')

    # Model.
    '''
    Let's build a small network with: 
        ->two convolutional layers
        ->one fully connected layer.
    '''
    def model(data):
        #1st Conv Layer
        conv1 = tf.nn.conv2d(input=data, filter=layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        print('Data: ',data)
        print('Conv1: ',conv1)
        print('Pool1: ',pool1)
        print('Hidden1: ',hidden1,'\n')
        
        #2nd Conv Layer
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(hidden2,[1,2,2,1],[1,2,2,1],padding='SAME')
        print('Conv2: ',conv2)
        print('Pool2: ',pool2)
        print('Hidden2: ',hidden2,'\n')
        
        
        #Fully connected layer
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        fullyConnected = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        print('FullyConnected: ',fullyConnected,'\n')
        
        return tf.matmul(fullyConnected, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

LAYER1 - Weights:  Tensor("Variable/read:0", shape=(5, 5, 1, 16), dtype=float32)
LAYER1 - Bias:  Tensor("Variable_1/read:0", shape=(16,), dtype=float32) 

LAYER2 - Weights:  Tensor("Variable_2/read:0", shape=(5, 5, 16, 16), dtype=float32)
LAYER2 - Bias:  Tensor("Variable_3/read:0", shape=(16,), dtype=float32) 

LAYER3 - Weights:  Tensor("Variable_4/read:0", shape=(784, 64), dtype=float32)
LAYER3 - Bias:  Tensor("Variable_5/read:0", shape=(64,), dtype=float32) 

LAYER4 - Weights:  Tensor("Variable_6/read:0", shape=(64, 10), dtype=float32)
LAYER4 - Bias:  Tensor("Variable_7/read:0", shape=(10,), dtype=float32) 

Data:  Tensor("Placeholder:0", shape=(16, 28, 28, 1), dtype=float32)
Conv1:  Tensor("Conv2D:0", shape=(16, 28, 28, 16), dtype=float32)
Pool1:  Tensor("MaxPool:0", shape=(16, 14, 14, 16), dtype=float32)
Hidden1:  Tensor("Relu:0", shape=(16, 28, 28, 16), dtype=float32) 

Conv2:  Tensor("Conv2D_1:0", shape=(16, 14, 14, 16), dtype=float32)
Pool2:  Tensor("MaxPool_1:0", shape=(16, 7, 

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [30]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        # C1 input 28 x 28
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)
        # S2 input 24 x 24
        pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # C3 input 12 x 12
        conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
        bias3 = tf.nn.relu(conv3 + layer2_biases)
        # S4 input 8 x 8
        pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        # F6 input 4 x 4
        shape = pool4.get_shape().as_list()
        reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))